In [22]:
import json

with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [23]:
docs_raw

[{'course': 'running-assistant-rag',
  'documents': [{'text': 'Interval training is a training method in which phases of intense effort alternate with recovery phases. This alternation between high and low intensity effectively improves endurance and speed, as the body is challenged both anaerobically and aerobically.',
    'section': 'Training',
    'question': 'What is interval training?'},
   {'text': "Long, slow runs are a central component of endurance training. They help improve basic endurance, strengthen the cardiovascular system, and increase the body's ability to burn fat. These runs also help prepare muscles and tendons for longer periods of exertion.",
    'section': 'Training',
    'question': 'What are long, slow runs good for?'},
   {'text': 'Recovery is an essential part of the training process. During recovery phases, the body can recover from training stimuli, repair muscle tissue, and adapt to the exertion. Without sufficient recovery, the risk of injury, overuse, an

In [24]:
documents = docs_raw[0]['documents']

In [25]:
documents

[{'text': 'Interval training is a training method in which phases of intense effort alternate with recovery phases. This alternation between high and low intensity effectively improves endurance and speed, as the body is challenged both anaerobically and aerobically.',
  'section': 'Training',
  'question': 'What is interval training?'},
 {'text': "Long, slow runs are a central component of endurance training. They help improve basic endurance, strengthen the cardiovascular system, and increase the body's ability to burn fat. These runs also help prepare muscles and tendons for longer periods of exertion.",
  'section': 'Training',
  'question': 'What are long, slow runs good for?'},
 {'text': 'Recovery is an essential part of the training process. During recovery phases, the body can recover from training stimuli, repair muscle tissue, and adapt to the exertion. Without sufficient recovery, the risk of injury, overuse, and decreased performance increases.',
  'section': 'Recovery',
  

In [26]:
import hashlib

def generate_document_id(doc):
    # combined = f"{doc['course']}-{doc['question']}"
    combined = f"{doc['question']}-{doc['text'][-10:]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id

In [27]:
for doc in documents:
    doc['id'] = generate_document_id(doc)

In [28]:
documents[1]

{'text': "Long, slow runs are a central component of endurance training. They help improve basic endurance, strengthen the cardiovascular system, and increase the body's ability to burn fat. These runs also help prepare muscles and tendons for longer periods of exertion.",
 'section': 'Training',
 'question': 'What are long, slow runs good for?',
 'id': 'c4c003a1'}

In [8]:
documents[3]['text'][-10:]

'g trinken.'

In [9]:
from collections import defaultdict

In [29]:
hashes = defaultdict(list)

for doc in documents:
    doc_id = doc['id']
    hashes[doc_id].append(doc)

In [30]:
hashes

defaultdict(list,
            {'1ce0afbb': [{'text': 'Interval training is a training method in which phases of intense effort alternate with recovery phases. This alternation between high and low intensity effectively improves endurance and speed, as the body is challenged both anaerobically and aerobically.',
               'section': 'Training',
               'question': 'What is interval training?',
               'id': '1ce0afbb'}],
             'c4c003a1': [{'text': "Long, slow runs are a central component of endurance training. They help improve basic endurance, strengthen the cardiovascular system, and increase the body's ability to burn fat. These runs also help prepare muscles and tendons for longer periods of exertion.",
               'section': 'Training',
               'question': 'What are long, slow runs good for?',
               'id': 'c4c003a1'}],
             '0c19360d': [{'text': 'Recovery is an essential part of the training process. During recovery phases, the 

In [31]:
len(hashes), len(documents)

(98, 98)

In [13]:
for k, values in hashes.items():
    if len(values) > 1:
        print(k, len(values))

In [81]:
# hashes['ff915831']

In [32]:
with open('docs-with-ids.json', 'wt', encoding='utf-8') as f_out:
    json.dump(documents, f_out, indent=2)

In [33]:
!head docs-with-ids.json

[
  {
    "text": "Interval training is a training method in which phases of intense effort alternate with recovery phases. This alternation between high and low intensity effectively improves endurance and speed, as the body is challenged both anaerobically and aerobically.",
    "section": "Training",
    "question": "What is interval training?",
    "id": "1ce0afbb"
  },
  {
    "text": "Long, slow runs are a central component of endurance training. They help improve basic endurance, strengthen the cardiovascular system, and increase the body's ability to burn fat. These runs also help prepare muscles and tendons for longer periods of exertion.",
    "section": "Training",


In [20]:
with open('docs-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

In [34]:
!head docs-with-ids.json

[
  {
    "text": "Interval training is a training method in which phases of intense effort alternate with recovery phases. This alternation between high and low intensity effectively improves endurance and speed, as the body is challenged both anaerobically and aerobically.",
    "section": "Training",
    "question": "What is interval training?",
    "id": "1ce0afbb"
  },
  {
    "text": "Long, slow runs are a central component of endurance training. They help improve basic endurance, strengthen the cardiovascular system, and increase the body's ability to burn fat. These runs also help prepare muscles and tendons for longer periods of exertion.",
    "section": "Training",


In [35]:
prompt_template = """
You emulate a runner who's asking questions about running techniques, training plans, recovery, and more.
Formulate 5 questions this runner might ask based on a FAQ record. The record
should contain the answer to the questions, and the questions should be complete and not too short.
If possible, use as fewer words as possible from the record. 

The record:

section: {section}
question: {question}
answer: {text}

Provide the output in parsable JSON without using code blocks:

["question1", "question2", ..., "question5"]
""".strip()

In [36]:
print(prompt_template.format(**documents[2]))

You emulate a runner who's asking questions about running techniques, training plans, recovery, and more.
Formulate 5 questions this runner might ask based on a FAQ record. The record
should contain the answer to the questions, and the questions should be complete and not too short.
If possible, use as fewer words as possible from the record. 

The record:

section: Recovery
question: Why is recovery important in running?
answer: Recovery is an essential part of the training process. During recovery phases, the body can recover from training stimuli, repair muscle tissue, and adapt to the exertion. Without sufficient recovery, the risk of injury, overuse, and decreased performance increases.

Provide the output in parsable JSON without using code blocks:

["question1", "question2", ..., "question5"]


In [37]:
from openai import OpenAI
client = OpenAI()

In [38]:
prompt = prompt_template.format(**documents[2])

response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": prompt}]
)

json_response = response.choices[0].message.content

In [39]:
json.loads(json_response)

['What does recovery allow the body to do after training?',
 'How does insufficient recovery affect performance?',
 'What role does recovery play in muscle repair?',
 'Why is avoiding overuse important in a training routine?',
 'What risks are associated with inadequate recovery periods?']

In [40]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )

    json_response = response.choices[0].message.content
    return json_response

In [41]:
from tqdm.auto import tqdm

/home/codespace/.local/share/virtualenvs/running-assistant-rag-j-iIWklV/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [42]:
results = {}

In [43]:
for doc in tqdm(documents): 
    doc_id = doc['id']
    if doc_id in results:
        continue

    questions = generate_questions(doc)
    results[doc_id] = questions

  0%|          | 0/98 [00:00<?, ?it/s]

100%|██████████| 98/98 [03:42<00:00,  2.27s/it]


In [44]:
results

{'1ce0afbb': '[\n    "How does alternating between intense and recovery phases benefit a runner?",\n    "Can combining high and low intensity in workouts enhance both aerobic and anaerobic capabilities?",\n    "What are the benefits of challenging your body with varying intensity levels during a run?",\n    "Why might a runner consider incorporating phases of intense effort and rest in their training?",\n    "How can alternating effort levels in training improve a runner\'s endurance and speed?"\n]',
 'c4c003a1': '[\n  "How do long, slow runs benefit endurance training?",\n  "Why are long, slow runs important for the cardiovascular system?",\n  "What role do long, slow runs play in fat-burning?",\n  "How do long, slow runs prepare muscles and tendons?",\n  "What advantages do long, slow runs offer for basic endurance?"\n]',
 '0c19360d': '[\n  "How does recovery aid the process of muscle repair in runners?",\n  "What are the potential risks of neglecting sufficient recovery time in a ru

In [46]:
json.loads(results['1ce0afbb'])

['How does alternating between intense and recovery phases benefit a runner?',
 'Can combining high and low intensity in workouts enhance both aerobic and anaerobic capabilities?',
 'What are the benefits of challenging your body with varying intensity levels during a run?',
 'Why might a runner consider incorporating phases of intense effort and rest in their training?',
 "How can alternating effort levels in training improve a runner's endurance and speed?"]

In [48]:
parsed_results = {}

for doc_id, json_questions in results.items():
    parsed_results[doc_id] = json.loads(json_questions)

In [49]:
parsed_results

{'1ce0afbb': ['How does alternating between intense and recovery phases benefit a runner?',
  'Can combining high and low intensity in workouts enhance both aerobic and anaerobic capabilities?',
  'What are the benefits of challenging your body with varying intensity levels during a run?',
  'Why might a runner consider incorporating phases of intense effort and rest in their training?',
  "How can alternating effort levels in training improve a runner's endurance and speed?"],
 'c4c003a1': ['How do long, slow runs benefit endurance training?',
  'Why are long, slow runs important for the cardiovascular system?',
  'What role do long, slow runs play in fat-burning?',
  'How do long, slow runs prepare muscles and tendons?',
  'What advantages do long, slow runs offer for basic endurance?'],
 '0c19360d': ['How does recovery aid the process of muscle repair in runners?',
  'What are the potential risks of neglecting sufficient recovery time in a running routine?',
  "In what ways does ade

In [ ]:
doc_index = {d['id']: d for d in documents}

{'1ce0afbb': {'text': 'Interval training is a training method in which phases of intense effort alternate with recovery phases. This alternation between high and low intensity effectively improves endurance and speed, as the body is challenged both anaerobically and aerobically.',
  'section': 'Training',
  'question': 'What is interval training?',
  'id': '1ce0afbb'},
 'c4c003a1': {'text': "Long, slow runs are a central component of endurance training. They help improve basic endurance, strengthen the cardiovascular system, and increase the body's ability to burn fat. These runs also help prepare muscles and tendons for longer periods of exertion.",
  'section': 'Training',
  'question': 'What are long, slow runs good for?',
  'id': 'c4c003a1'},
 '0c19360d': {'text': 'Recovery is an essential part of the training process. During recovery phases, the body can recover from training stimuli, repair muscle tissue, and adapt to the exertion. Without sufficient recovery, the risk of injury,

In [53]:
doc_index

{'1ce0afbb': {'text': 'Interval training is a training method in which phases of intense effort alternate with recovery phases. This alternation between high and low intensity effectively improves endurance and speed, as the body is challenged both anaerobically and aerobically.',
  'section': 'Training',
  'question': 'What is interval training?',
  'id': '1ce0afbb'},
 'c4c003a1': {'text': "Long, slow runs are a central component of endurance training. They help improve basic endurance, strengthen the cardiovascular system, and increase the body's ability to burn fat. These runs also help prepare muscles and tendons for longer periods of exertion.",
  'section': 'Training',
  'question': 'What are long, slow runs good for?',
  'id': 'c4c003a1'},
 '0c19360d': {'text': 'Recovery is an essential part of the training process. During recovery phases, the body can recover from training stimuli, repair muscle tissue, and adapt to the exertion. Without sufficient recovery, the risk of injury,

In [52]:
final_results = []

for doc_id, questions in parsed_results.items():
    for q in questions:
        final_results.append((q, doc_id))

In [54]:
final_results

[('How does alternating between intense and recovery phases benefit a runner?',
  '1ce0afbb'),
 ('Can combining high and low intensity in workouts enhance both aerobic and anaerobic capabilities?',
  '1ce0afbb'),
 ('What are the benefits of challenging your body with varying intensity levels during a run?',
  '1ce0afbb'),
 ('Why might a runner consider incorporating phases of intense effort and rest in their training?',
  '1ce0afbb'),
 ("How can alternating effort levels in training improve a runner's endurance and speed?",
  '1ce0afbb'),
 ('How do long, slow runs benefit endurance training?', 'c4c003a1'),
 ('Why are long, slow runs important for the cardiovascular system?',
  'c4c003a1'),
 ('What role do long, slow runs play in fat-burning?', 'c4c003a1'),
 ('How do long, slow runs prepare muscles and tendons?', 'c4c003a1'),
 ('What advantages do long, slow runs offer for basic endurance?', 'c4c003a1'),
 ('How does recovery aid the process of muscle repair in runners?',
  '0c19360d'),


In [55]:
import pandas as pd

df = pd.DataFrame(final_results, columns=['question', 'document'])

In [56]:
df.head()

,question,document
0,How does alternating between intense and recov...,1ce0afbb
1,Can combining high and low intensity in workou...,1ce0afbb
2,What are the benefits of challenging your body...,1ce0afbb
3,Why might a runner consider incorporating phas...,1ce0afbb
4,How can alternating effort levels in training ...,1ce0afbb


In [57]:
df.to_csv('ground-truth-data.csv', index=False)